# SQL Lab

Congratulations! We've been hired as analysts at Classic Model Car Company, and you've been handed keys to the Data Warehouse.  They want you to hit the ground running with analysis! We have lots of questions to answer.  First, let's connect to the database.

1. Connect via MySQL Credentials
  - hostname: `hbs-ds-060120.c1doesqrid0e.us-east-1.rds.amazonaws.com`
  - username: `hbs-student`
  - password: (distributed via slack)
  - port: 3306 (default)
  - database: `classicmodels`

2. Take a moment to study the schema, noting the keys. Browse the data in each relation if you wish. Answer the following queries. I've given you the expected results below each query.

3. Answer the following questions! Do them in your MySQL client first, most likely, then bring them into Pandas here in this notebook.

1. List all customer names and their addresses who don't have a specified sales rep. Order by customer name.

In [ ]:
!pip install MySQL-python

In [2]:
import pandas as pd
import mysql.connector

con = mysql.connector.connect(user='hbs-student', password='super-secure-password?',
                              host='hbs-ds-060120.c1doesqrid0e.us-east-1.rds.amazonaws.com',
                              database='classicmodels')
con.close()

In [ ]:


pd.read_sql_query("""
-- List all customer names and their addresses who don't have a
-- specified sales rep. Order by customer name.

SELECT 
contactLastName,
contactFirstName,
addressline1,
addressline2,
city,state,
postalCode,
country
FROM classicmodels.customers
WHERE salesRepEmployeeNumber IS NULL
ORDER BY contactLastName ASC
 """)

2. List the names of all customers that have a creditLimit between 100,000 and 200,000 inclusively. Order by customer name.

In [ ]:


pd.read_sql_query("""
SELECT 
contactLastName,
contactFirstName,
creditLimit
FROM classicmodels.customers
WHERE creditLimit>=100000 AND creditLimit<=200000
ORDER BY contactLastName ASC;
 """)


3. List all employees who have the initials of M.P. or P.M., by name. Order by last name.

In [ ]:
pd.read_sql_query("""
SELECT 
lastName,
firstName,
SUBSTRING(firstName,1,1) AS first_initial,
SUBSTRING(lastName,1,1) AS last_initial
FROM classicmodels.employees
HAVING first_initial='M' AND last_initial='P' OR first_initial='P' AND last_initial='M'
ORDER BY lastName ASC;
 """)

4. List all the products' productNames purchased by Mini Wheels Co. Order the list by product line.

In [ ]:
pd.read_sql_query("""
SELECT 
productName,
customerName,
productLine
FROM products
JOIN orderdetails USING (productCode)
JOIN orders USING (orderNumber)
JOIN customers USING (customerNumber)
WHERE customerName='Mini Wheels Co.'
ORDER BY productLine
""")

5. List the customers by customerName whose contact person shares either a first name or last name with any of the company's employees Order by customerName

In [ ]:
pd.read_sql_query("""
SELECT
customerName,
contactFirstName,
contactLastName,
employees.firstName,
employees.lastName
FROM customers
JOIN employees 
ON customers.salesRepEmployeeNumber = employees.employeeNumber
WHERE contactFirstName=employees.firstName OR contactLastName=employees.lastName
ORDER BY customerName

""")

7. List the customers (by name) that are based in the country which is first in the lexicographic order of customer countries. Order by customerName

In [ ]:
pd.read_sql_query("""
SELECT 
customerName,country
FROM customers
WHERE country=(SELECT country FROM customers
ORDER BY country ASC
LIMIT 1)
ORDER BY customerName
""")

8. Identify the product(s) by productCode and productName that the company has the least stock of. Order by product code.

In [ ]:
pd.read_sql_query("""
SELECT 
productName,
productCode,
quantityInStock
FROM products
WHERE quantityInStock<1000
ORDER BY productCode
""")

9. Identify the offices (by city) that have the least employees and the most employees (answer should include city, numEmps)

In [ ]:
pd.read_sql_query("""
SELECT 
city,
COUNT(employeeNumber) as numEmps
FROM offices
JOIN employees USING (officeCode)
GROUP BY city
ORDER BY numEmps
""")

10. For all Sales Reps list their names (first and last) and the name of the office they work out of and the number of clients they represent. Order by reverse order of numClients

In [ ]:
pd.read_sql_query("""
SELECT 
firstName,
lastName,
offices.city,
COUNT(customerNumber) as numClients
FROM offices
JOIN employees USING (officeCode)
JOIN customers
ON salesRepEmployeeNumber=employeeNumber
GROUP BY salesRepEmployeeNumber
ORDER BY numClients ASC
""")

11. List the customer(s) and their total payments (answer should include customerNumber, customerName, totalPayment where totalPayment is the total amount of dollars the customer has paid). Order by totalPayment.

In [ ]:
pd.read_sql_query("""
SELECT 
customerNumber,
customerName,
SUM(amount) as totalPayment
FROM customers
JOIN payments USING (customerNumber)
GROUP BY customerNumber
ORDER BY totalPayment DESC
""")

12. List the product number, product name, and customer name such that the customer has never ordered that product. Order by customerName, productCode.

In [ ]:
pd.read_sql_query("""
SELECT
productCode,
productName,
customerName
FROM products, customers
WHERE (productCode, customerNumber) NOT IN
(SELECT productCode, customerNumber
FROM orders
JOIN orderdetails USING (orderNumber))
ORDER BY customerName, productCode
""")

13. List the profit per product that the company would make if it were able to sell all of that product at MSRP (answer should include productName, profit). Order by profit.

In [ ]:
pd.read_sql_query("""
SELECT 
productName,
(MSRP - buyPrice) * quantityinStock as profit
FROM products
ORDER BY profit DESC
""")

14. List the average order size for each customer (answer should include customerName, avgQuantity) Order by customer name.

In [ ]:
pd.read_sql_query("""
SELECT 
customerName, 
AVG(orderSize) AS avgQuantity
FROM(
SELECT customerName, 
SUM(quantityOrdered) AS orderSize
FROM customers NATURAL JOIN orders NATURAL JOIN orderdetails
GROUP BY orders.ordernumber
) AS totalOrder
GROUP BY customerName
ORDER BY customerName
""")